In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

In [ ]:
labels=pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')

In [ ]:
type(labels)

In [ ]:
labels.id = labels.id.astype('str') + '.jpg'

In [ ]:
df_train, df_val_test= train_test_split(labels, test_size=0.2, random_state=42)
df_test, df_val= train_test_split(df_val_test, test_size=0.5, random_state=42)

In [ ]:
Image_Width=224
Image_Height=224
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size=8

In [ ]:
%cd ../input/dog-breed-identification
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, fill_mode="nearest")
train_generator = train_datagen.flow_from_dataframe(df_train, directory="./train/",x_col='id',y_col='breed', target_size=Image_Size, class_mode='categorical', batch_size=batch_size,)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(df_val,  "./train/",x_col='id',y_col='breed',target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(df_test,  "./train/",x_col='id',y_col='breed',target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

In [ ]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K


class ResNet:
    @staticmethod

    def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        # the shortcut branch of the ResNet module should be initialize as the input (identity) data
        shortcut = data
        # the first block of the ResNet module are the 1x1 CONVs
        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
        act1 = Activation("relu")(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act1)

        # the second block of the ResNet module are the 3x3 CONVs
        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
        act2 = Activation("relu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same",
                       use_bias=False, kernel_regularizer=l2(reg))(act2)

        # the third block of the ResNet module is another set of 1x1 CONVs
        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
        act3 = Activation("relu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False,
                       kernel_regularizer=l2(reg))(act3)

        # if we are to reduce the spatial size, apply a CONV layer to the shortcut
        if red:
            shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False,
                              kernel_regularizer=l2(reg))(act1)

        # add together the shortcut and the final CONV
        x = add([conv3, shortcut])

        # return the addition as the output of the ResNet module
        return x

    @staticmethod
    def build(width, height, depth, classes, stages, filters, reg=0.0001,
              bnEps=2e-5, bnMom=0.9, dataset="cifar"):

        # initialize the input shape to be "channels last" and the channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # set the input and apply BN
        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum = bnMom)(inputs)

        # check if we are utilizing the CIFAR dataset
        if dataset == "cifar":
            # apply a single CONV layer
            x = Conv2D(filters[0], (3, 3), use_bias=False, padding = "same",
                       kernel_regularizer = l2(reg))(x)

        # loop over the number of stages
        for i in range(0, len(stages)):
        # initialize the stride, then apply a residual module
        # used to reduce the spatial size of the input volume
            stride = (1, 1) if i == 0 else (2, 2)
            x = ResNet.residual_module(x, filters[i + 1], stride, chanDim,
                                       red = True, bnEps = bnEps, bnMom = bnMom)

            # loop over the number of layers in the stage
            for j in range(0, stages[i] - 1):
            # apply a ResNet module
                x = ResNet.residual_module(x, filters[i + 1],(1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        # apply BN => ACT => POOL
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum = bnMom)(x)
        x = Activation("relu")(x)
        x = AveragePooling2D((8, 8))(x)

        # softmax classifier
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax")(x)

        # create the model
        model = Model(inputs, x, name="resnet")

        # return the constructed network architecture
        return model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.datasets import cifar10
from keras.models import load_model
import keras.backend as K
import numpy as np
import argparse
import sys
from sklearn.preprocessing import LabelBinarizer
import matplotlib
matplotlib.use("Agg")
print("[INFO] compiling model...")
opt = keras.optimizers.SGD()
model = ResNet.build(224, 224, 3, 120, (9, 9, 9),(64, 64, 128, 256), reg=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
checkpoint=ModelCheckpoint('/kaggle/working/model1.h5',
                           monitor="val_accuracy",
                           verbose=0,
                           save_best_only=True,
                           save_weights_only=False,
                           mode="auto",
                           save_freq="epoch")


def step_decay(epoch):
    # initialize the base initial learning rate, drop factor, and
    # epochs to drop every
    initAlpha = 0.2
    decay_rate = 0.005

    # compute learning rate for the current epoch
    alpha = initAlpha * 1/(1 + decay_rate * epoch)

    # return the learning rate
    return float(alpha)


callbacks = [LearningRateScheduler(step_decay),checkpoint]

In [ ]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [ ]:
epochs = 64
batch_size=8
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(df_val)//batch_size,
    steps_per_epoch=len(df_train)//batch_size,
    callbacks=callbacks
)
